## Imports & Job Parameters

These get adjusted based upon how many records you want per chunk, desired CRS, etc...

In [6]:
import requests
import json
#import csv
import socket
#from datetime import datetime
#from itertools import islice
import pandas as pd

#Input csv file
#inCSV = open("testAddresses.csv")
inCSV = "testAddresses.csv"
# output spatial reference id 
outsrid = '4326'
# Total number of records to submit
sliceSize = 20
# Number of records to submit per GET request
chunkSize = 5

## Token request

In [ ]:
# api-endpoint for requesting a token from locator.stanford.edu

tokenURL = "https://locator.stanford.edu/arcgis/tokens/generateToken"
payload = "username=win%5C<your SUNetID>&password=<your password>%40&client=requestip&expiration=86400&f=json"
headers = {
    'Content-Type': "application/x-www-form-urlencoded",
    'User-Agent': "PostmanRuntime/7.13.0",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Postman-Token': "2686259f-3e03-44d3-920e-abd43ae0c908,b4f42197-a2ae-4ef3-9dca-c0f5932bb21b",
    'Host': "locator.stanford.edu",
    'accept-encoding': "gzip, deflate",
    'content-length': "81",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }
# request token from locator.stanford.edu/arcgis/tokens using REST POST
mytoken = requests.post( tokenURL, data=payload, headers=headers).json()['token']

## Alternatively: request token at https://locator.stanford.edu/arcgis/tokens/generateToken

In [3]:
# set token manually
mytoken = "Vy4qADiBl8huHLW7zcSyoAqvtl0nkOfAihktgGbza0WzvIUkuBxIsXVO0_bI9iqb"

## Parameters for the geocoding REST POST request

In [4]:
# api-endpoint for geocoding US Street Addresses with locator.stanford.edu

url = 'https://locator.stanford.edu/arcgis/rest/services/geocode/USA_StreetAddress/GeocodeServer/geocodeAddresses'

#output format
format = 'json'

# header
header = {'Content-Type': 'application/x-www-form-urlencoded'}

# params data to be sent to api 
params ={'outSR':outsrid,'f':format,'token':mytoken}

## Make a new N record CSV from the oneMillionAddresses.csv then chunk & submit geocodes to locator.stanford.edu, then write response to new csv

In [7]:
#Job startTime
jobStartTime = datetime.now()

# open outCSV in write mode, so it will override existing outfiles
with open('outCSV.csv', 'w') as outf:
    chunk_df = pd.DataFrame([])
    
    # read slice in with correct data types and write it to slicefile
    for slice in pd.read_csv(inCSV, chunksize=sliceSize, dtype={
            "OBJECTID" : int,
            "ZIP" : str
        }):
        slice.to_csv('slicedAddresses.csv', index=False)
    
        # now process chunks from slice and corece to correct data types
        for chunk in pd.read_csv('slicedAddresses.csv', chunksize=chunkSize, dtype={
            "OBJECTID" : int,
            "ZIP" : str
            }):
            # make it json 
            chunk_dict =  chunk.to_dict('records')       
            # Nest each record under an attribute object, iterating through the dictionary           
            attribute = [{'attributes': row } for row in chunk_dict]
            addresses = json.dumps({'records':attribute}) 
        
            # send it to geocoding
            r = requests.post(url, headers = header, data = { 'addresses':addresses,'outSR':outsrid,'f':format,'token':mytoken})
            parsedJson = json.loads(r.text)

            # convert json response back to tabular
            for row in pd.DataFrame(parsedJson["locations"])['attributes']:
                r = pd.DataFrame(data=row, index=[0])
                chunk_df = chunk_df.append(r)          
            
    # append to outCSV table        
    chunk_df.to_csv(outf, mode='a', index=False, header=not outf.tell())

jobEndTime = datetime.now()    
print("Finished in: "+str(jobEndTime-jobStartTime))  

Finished in: 0:00:01.504678
